# Hackaton orienté santé

On se demande si ...  
Peut-on établir un lien entre la densité de médecins par spécialité  et par territoire et la pratique du dépassement d'honoraires?   
Est-ce  dans les territoires où la densité est la plus forte que les médecins  pratiquent le moins les dépassement d'honoraires ?   
Est ce que la densité de certains médecins / praticiens est corrélé à la densité de population pour certaines classes d'ages (bebe/pediatre, personnes agées / infirmiers etc...) ?

## Import et collecte des données

In [336]:
import pandas as pd
# graph

import plotly
plotly.tools.set_credentials_file(username='telecommsbgd2', api_key='vycTMoJ16qDZEdUWiOiC')
from plotly import tools
import plotly.figure_factory as ff
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

**Collecte des informations de santé**

In [337]:
sheetlist= pd.ExcelFile("Honoraires_totaux_des_professionnels_de_sante_par_departement_en_2016.xls")
print(sheetlist.sheet_names)
exclues=['Lisez moi', 'Nomenclature des PS','Laboratoires','Spécialistes']

df = sheetlist.parse('Spécialistes')
df.rename(columns = {list(df)[0]:'typesR'}, inplace=True)
df.columns = df.columns.map(lambda x: str(x).replace('(Euros)','').replace(" ","_"))
print(df.describe())
for sheet in sheetlist.sheet_names:
    if sheet not in exclues:
        tmp = sheetlist.parse(sheet)
        tmp.rename(columns = {list(tmp)[0]:'typesR'}, inplace=True)
        tmp.columns = tmp.columns.map(lambda x: str(x).replace('(Euros)','').replace(" ","_"))        
        print(tmp.shape)
        df = df.append(tmp)
        #df = df.merge(tmp, left_on=['DEPARTEMENT'],right_on=[str('DEPARTEMENT'+sheet).replace(" ","_")], how='inner')
df['DEP_num'] = df.DEPARTEMENT.str.extract(r'(\d[\dA-Z])')
df.drop(df.query('DEPASSEMENTS_ == "nc"').index,axis=0,inplace=True)
df.DEPASSEMENTS_ = pd.to_numeric(df.DEPASSEMENTS_.astype(float) )
print(df.shape)
df.head()

['Lisez moi', 'Nomenclature des PS', 'Spécialistes', 'Généralistes et MEP', 'Dentistes et ODF', 'Sages-femmes', 'Auxiliaires médicaux', 'Laboratoires']
           EFFECTIFS
count    5512.000000
mean      137.378991
std      2425.039230
min         0.000000
25%         1.000000
50%         5.000000
75%        20.000000
max    114542.000000
(6968, 7)
(520, 7)
(104, 7)
(624, 7)
(7018, 8)


,typesR,DEPARTEMENT,EFFECTIFS,HONORAIRES_SANS_DEPASSEMENT_,DEPASSEMENTS_,FRAIS_DE_DEPLACEMENT_,TOTAL_DES_HONORAIRES_,DEP_num
0,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4.53008e+06,2496180.45,0,7.02626e+06,01
1,02- Anesthésie-réanimation chirurgicale,02- Aisne,19,4.98526e+06,373837.57,0,5.3591e+06,02
2,02- Anesthésie-réanimation chirurgicale,03- Allier,19,5.1029e+06,560978.59,0,5.66388e+06,03
5,02- Anesthésie-réanimation chirurgicale,06- Alpes-Maritimes,110,2.48538e+07,10816346.67,0,3.56701e+07,06
6,02- Anesthésie-réanimation chirurgicale,07- Ardèche,23,5.27125e+06,802542.01,0,6.07379e+06,07


On a bien le bon nombre de ligne attendu à la fin de la lecture du fichier excel on peut donc supposer que son ouverture c'est bien déroulé  
**Collecte des informations de latitude et longitude des villes françaises**

In [338]:
region = pd.read_csv("EUCircos_Regions_departements_circonscriptions_communes_gps.csv",sep=';',decimal=',',na_values='')
region['DEPARTEMENT']=region.numéro_département.map(lambda x : x if len(x)>1 else str('0'+x))
region.latitude = pd.to_numeric(region.latitude,errors='coerce')
region.longitude = pd.to_numeric(region.longitude,errors='coerce')
region.head()

,EU_circo,code_région,nom_région,chef-lieu_région,numéro_département,nom_département,préfecture,numéro_circonscription,nom_commune,codes_postaux,code_insee,latitude,longitude,éloignement,DEPARTEMENT
0,Sud-Est,82,Rhône-Alpes,Lyon,1,Ain,Bourg-en-Bresse,1,Attignat,1340,1024,46.283333,5.166667,1.21,01
1,Sud-Est,82,Rhône-Alpes,Lyon,1,Ain,Bourg-en-Bresse,1,Beaupont,1270,1029,46.400000,5.266667,1.91,01
2,Sud-Est,82,Rhône-Alpes,Lyon,1,Ain,Bourg-en-Bresse,1,Bény,1370,1038,46.333333,5.283333,1.51,01
3,Sud-Est,82,Rhône-Alpes,Lyon,1,Ain,Bourg-en-Bresse,1,Béreyziat,1340,1040,46.366667,5.050000,1.71,01
4,Sud-Est,82,Rhône-Alpes,Lyon,1,Ain,Bourg-en-Bresse,1,Bohas-Meyriat-Rignat,1250,1245,46.133333,5.400000,1.01,01


In [339]:
dfregion=region[['latitude','longitude','DEPARTEMENT']].groupby(by="DEPARTEMENT").agg({'latitude':'median','longitude':'median'})

**Jointure avec des lattitudes/longitudes avec la table d'origine**

In [340]:
df = df.merge(dfregion,right_on='DEPARTEMENT',left_on='DEP_num',how='left').query('latitude != "NaN"')
print(df.shape)
df.head(20)

(6428, 10)


,typesR,DEPARTEMENT,EFFECTIFS,HONORAIRES_SANS_DEPASSEMENT_,DEPASSEMENTS_,FRAIS_DE_DEPLACEMENT_,TOTAL_DES_HONORAIRES_,DEP_num,latitude,longitude
0,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4.53008e+06,2496180.45,0,7.02626e+06,01,46.100000,5.333333
1,02- Anesthésie-réanimation chirurgicale,02- Aisne,19,4.98526e+06,373837.57,0,5.3591e+06,02,49.550000,3.483333
2,02- Anesthésie-réanimation chirurgicale,03- Allier,19,5.1029e+06,560978.59,0,5.66388e+06,03,46.333333,3.200000
3,02- Anesthésie-réanimation chirurgicale,06- Alpes-Maritimes,110,2.48538e+07,10816346.67,0,3.56701e+07,06,43.833333,7.116667
4,02- Anesthésie-réanimation chirurgicale,07- Ardèche,23,5.27125e+06,802542.01,0,6.07379e+06,07,44.750000,4.466667
5,02- Anesthésie-réanimation chirurgicale,09- Ariège,0,0,0.00,0,0,09,42.983333,1.583333
6,02- Anesthésie-réanimation chirurgicale,10- Aube,12,3.69362e+06,9543.92,0,3.70317e+06,10,48.300000,4.183333
7,02- Anesthésie-réanimation chirurgicale,11- Aude,17,4.49699e+06,313839.98,0,4.81083e+06,11,43.133333,2.283333
8,02- Anesthésie-réanimation chirurgicale,13- Bouches-du-Rhône,209,4.84844e+07,3000051.42,0,5.14844e+07,13,43.533333,5.250000
9,02- Anesthésie-réanimation chirurgicale,14- Calvados,42,1.38657e+07,304614.28,0,1.41703e+07,14,49.133333,-0.383333


In [341]:
import math
from ast import literal_eval
def merc(lat,lon):
    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    
    return (x, y)

df['latitudemerc'] = df[['latitude','longitude']].apply(lambda x: merc(x['latitude'], x['longitude'])[0],axis=1)
df['longitudemerc'] = df[['latitude','longitude']].apply(lambda x: merc(x['latitude'], x['longitude'])[1],axis=1)

**Collecte de la répartition de la population par tranche d'age**

In [342]:
sheetlist2= pd.ExcelFile("estim-pop-dep-sexe-gca-1975-2018.xls")
pop = sheetlist2.parse("2016",skiprows=4)
pop.columns=['DEP_num','DEP_txt','0/19E','20/39E','40/59E','60/74E','75+E','TotalE','0/19H','20/39H','40/59H','60/74H','75+H','TotalH','0/19F','20/39F','40/59F','60/74F','75+F','TotalF'] 
pop.describe()

,0/19E,20/39E,40/59E,60/74E,75+E,TotalE,0/19H,20/39H,40/59H,60/74H,75+H,TotalH,0/19F,20/39F,40/59F,60/74F,75+F,TotalF
count,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02,1.040000e+02
mean,4.729187e+05,4.607744e+05,5.109796e+05,3.032773e+05,1.759402e+05,1.923890e+06,2.418397e+05,2.278959e+05,2.510131e+05,1.435070e+05,6.723932e+04,9.314951e+05,2.310790e+05,2.328785e+05,2.599666e+05,1.597702e+05,1.087009e+05,9.923952e+05
std,2.195210e+06,2.152536e+06,2.384779e+06,1.421193e+06,8.267843e+05,8.979648e+06,1.122708e+06,1.065784e+06,1.172225e+06,6.725236e+05,3.158723e+05,4.348720e+06,1.072503e+06,1.086761e+06,1.212558e+06,7.486722e+05,5.109150e+05,4.630933e+06
min,1.624400e+04,1.504000e+04,2.118800e+04,7.740000e+03,2.256000e+03,7.607800e+04,8.364000e+03,7.564000e+03,1.093000e+04,4.092000e+03,1.055000e+03,3.785600e+04,7.880000e+03,7.476000e+03,1.025800e+04,3.648000e+03,1.201000e+03,3.822200e+04
25%,7.230650e+04,6.360850e+04,7.970300e+04,5.396075e+04,3.362850e+04,3.020062e+05,3.699675e+04,3.112225e+04,3.948000e+04,2.609100e+04,1.318350e+04,1.469480e+05,3.530975e+04,3.164525e+04,3.964350e+04,2.780450e+04,2.044500e+04,1.538192e+05
50%,1.326385e+05,1.149965e+05,1.421420e+05,9.050800e+04,4.995150e+04,5.379825e+05,6.706250e+04,5.762250e+04,7.025300e+04,4.282450e+04,1.938200e+04,2.625230e+05,6.550150e+04,5.737400e+04,7.188900e+04,4.745550e+04,3.087800e+04,2.760855e+05
75%,2.259705e+05,2.145038e+05,2.525905e+05,1.482462e+05,8.372875e+04,9.425485e+05,1.159748e+05,1.061162e+05,1.262592e+05,7.012375e+04,3.202800e+04,4.591460e+05,1.099958e+05,1.097815e+05,1.271472e+05,7.812250e+04,5.142750e+04,4.831558e+05
max,1.639451e+07,1.597351e+07,1.771396e+07,1.051361e+07,6.099262e+06,6.669486e+07,8.383776e+06,7.900392e+06,8.701787e+06,4.974911e+06,2.330963e+06,3.229183e+07,8.010738e+06,8.073121e+06,9.012174e+06,5.538702e+06,3.768299e+06,3.440303e+07


In [343]:
selection = ['DEP_num','0/19E','20/39E','40/59E','60/74E','75+E','0/19H','20/39H','40/59H',
             '60/74H','75+H','0/19F','20/39F','40/59F','60/74F','75+F']
pop = pop[selection].melt(id_vars='DEP_num')
pop['qte']=pd.to_numeric(pop.value)
pop.head()

,DEP_num,variable,value,qte
0,01,0/19E,167021.0,167021.0
1,02,0/19E,136884.0,136884.0
2,03,0/19E,71186.0,71186.0
3,04,0/19E,34812.0,34812.0
4,05,0/19E,31451.0,31451.0


In [344]:
selection2 = ['DEP_num','age','sexe','qte']
pop[['age','sexe']] = pop['variable'].str.extract(r'([0-9]+/[0-9]+|75\+)([A-Z])')
pop.drop(columns=['variable','value'],inplace=True)

**Jointure de la répartition de la population avec la table principale**

In [345]:
df = df.merge(pop,on="DEP_num",how='left')
df[['types_num','types_text']] = df.typesR.str.extract(r'(\d+)- ([\wéè ]+)')
df.dropna(axis=0,subset=['types_num','types_text'],inplace=True)
df.head()

,typesR,DEPARTEMENT,EFFECTIFS,HONORAIRES_SANS_DEPASSEMENT_,DEPASSEMENTS_,FRAIS_DE_DEPLACEMENT_,TOTAL_DES_HONORAIRES_,DEP_num,latitude,longitude,latitudemerc,longitudemerc,qte,age,sexe,types_num,types_text
0,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4.53008e+06,2496180.45,0,7.02626e+06,01,46.1,5.333333,593703.913791,5.796389e+06,167021.0,0/19,E,02,Anesthésie
1,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4.53008e+06,2496180.45,0,7.02626e+06,01,46.1,5.333333,593703.913791,5.796389e+06,147968.0,20/39,E,02,Anesthésie
2,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4.53008e+06,2496180.45,0,7.02626e+06,01,46.1,5.333333,593703.913791,5.796389e+06,177879.0,40/59,E,02,Anesthésie
3,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4.53008e+06,2496180.45,0,7.02626e+06,01,46.1,5.333333,593703.913791,5.796389e+06,94794.0,60/74,E,02,Anesthésie
4,02- Anesthésie-réanimation chirurgicale,01- Ain,17,4.53008e+06,2496180.45,0,7.02626e+06,01,46.1,5.333333,593703.913791,5.796389e+06,50205.0,75+,E,02,Anesthésie


On possede maintenant un jeu de données contenant les informations qu'on veux vérifier
## Data sniffing

lien entre la densité de médecins par spécialité  et par territoire et la pratique du dépassement d'honoraires

In [349]:

import numpy as np

from bokeh.layouts import row, widgetbox
from bokeh.models import CustomJS, Slider
from bokeh.models import ColumnDataSource, HoverTool 
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook

from bokeh.palettes import Viridis11
from bokeh.transform import linear_cmap


output_notebook()


x = np.linspace(0, 10, 500)
y = np.sin(x)

source = ColumnDataSource(data=dict(x=x, y=y))

plot = figure(y_range=(-10, 10), plot_width=400, plot_height=400)

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

callback = CustomJS(args=dict(source=source), code="""
    var data = source.data;
    var A = amp.value;
    var k = freq.value;
    var phi = phase.value;
    var B = offset.value;
    var x = data['x']
    var y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = B + A*Math.sin(k*x[i]+phi);
    }
    source.change.emit();
""")

amp_slider = Slider(start=0.1, end=10, value=1, step=.1,
                    title="Amplitude", callback=callback)
callback.args["amp"] = amp_slider

freq_slider = Slider(start=0.1, end=10, value=1, step=.1,
                     title="Frequency", callback=callback)
callback.args["freq"] = freq_slider

phase_slider = Slider(start=0, end=6.4, value=0, step=.1,
                      title="Phase", callback=callback)
callback.args["phase"] = phase_slider

offset_slider = Slider(start=-5, end=5, value=0, step=.1,
                       title="Offset", callback=callback)
callback.args["offset"] = offset_slider

layout = row(
    plot,
    widgetbox(amp_slider, freq_slider, phase_slider, offset_slider),
)



show(layout,notebook_handle=True)


Loading BokehJS ...

In [350]:
question1 = df.query("sexe == 'E'")[['typesR','EFFECTIFS','DEPASSEMENTS_','DEPARTEMENT','latitudemerc','longitudemerc']] \
.dropna(axis=0,subset=["DEPASSEMENTS_"])\
.query("DEPASSEMENTS_ != 0")\
.groupby(by=['DEPARTEMENT','typesR'])\
.agg({'EFFECTIFS':'mean','DEPASSEMENTS_':'sum','latitudemerc':'mean','longitudemerc':'mean'})
question1[['EFFECTIFS','DEPASSEMENTS_',]] /= df[['EFFECTIFS','DEPASSEMENTS_',]].max()
question1.EFFECTIFS=question1.EFFECTIFS*100
question1.DEPASSEMENTS_=question1.DEPASSEMENTS_*100
question1['type']=question1.index.values

In [351]:

from matplotlib import cm
cm.get_cmap('Greys')

In [357]:



# For GMaps to function, Google requires you obtain and enable an API key:
#
#     https://developers.google.com/maps/documentation/javascript/get-api-key
#
# Replace the value below with your personal API key:

from bokeh.tile_providers import CARTODBPOSITRON
p = figure(x_range=(-750000, 1000000), y_range=(4800000, 6793952),tools=[hover, 'wheel_zoom','save','pan'])

source = ColumnDataSource(
    data=dict(lat=question1.latitudemerc,
              lon=question1.longitudemerc,
              sze=question1.EFFECTIFS,
             clr=question1.DEPASSEMENTS_,
             tpe=question1.type)
)
hover = HoverTool(tooltips=[
    ("type", "@tpe")
    
])

mapper = linear_cmap(field_name='clr', palette=Viridis11 ,low=min(question1.DEPASSEMENTS_) ,high=max(question1.DEPASSEMENTS_))
p.add_tile(CARTODBPOSITRON)


p.circle(x="lat", y="lon", size="sze", fill_color=mapper, fill_alpha=0.5,line_alpha=0, source=source)

show(p)